# 3. Discriminant Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tom-gemini-cloud/machine_learning_practice/blob/main/ml_week_2/discriminant_analysis.ipynb)


## Discriminant Analysis

Discriminant Analysis is a family of supervised classification methods that model each class's distribution and then use Bayes' rule to assign the most likely class.

### The Core Idea

Assume each data point is a feature vector:

- $\mathbf{x} \in \mathbb{R}^d$ (a vector of $d$ real-valued features)

Assume the features come from a **class-conditional density**:

- $p(\mathbf{x} \mid y = k)$ (the probability density of seeing $\mathbf{x}$ if the class is $k$)

and each class has a **prior probability**:

- $\pi_k = P(y = k)$ (how common class $k$ is overall)

> **Notation:** Lowercase $p$ denotes a probability _density_ (for continuous variables like $\mathbf{x}$), while uppercase $P$ denotes a probability of a discrete event (like the class label $y = k$). Densities can exceed 1, but probabilities are always between 0 and 1.

### Prediction rule

Predict the class $\hat{y}$ by choosing the class $k$ that maximises:

$$
\hat{y} = \arg\max_k \left[ \log p(\mathbf{x} \mid y = k) + \log \pi_k \right]
$$

**In plain English:** For each possible class, compute a score by adding two things together:

1. **How well the data fits that class** — $\log p(\mathbf{x} \mid y = k)$ measures how likely we'd see this particular data point if it truly belonged to class $k$
2. **How common that class is** — $\log \pi_k$ gives a "bonus" to classes that appear more frequently in the training data

Then pick whichever class has the highest total score. The $\arg\max_k$ simply means "find the value of $k$ that gives the maximum".

We use logarithms because they turn multiplication into addition (making computation easier) and don't change which class wins (since log is monotonically increasing).

### Common modelling choice

Most commonly, $p(\mathbf{x} \mid y = k)$ is modelled as a **multivariate Gaussian**.


## Linear Discriminant Analysis (LDA)

LDA assumes that all classes share the **same covariance matrix** $\Sigma$. Each class $k$ has its own mean $\boldsymbol{\mu}_k$, but the shape and orientation of the distribution is identical across classes.

### The Model

For each class $k$, the class-conditional density is:

$$
p(\mathbf{x} \mid y = k) = \frac{1}{(2\pi)^{d/2} |\Sigma|^{1/2}} \exp\left( -\frac{1}{2} (\mathbf{x} - \boldsymbol{\mu}_k)^\top \Sigma^{-1} (\mathbf{x} - \boldsymbol{\mu}_k) \right)
$$

### The Discriminant Function

Taking the log and dropping terms that don't depend on $k$, we get the **linear discriminant function**:

$$
\delta_k(\mathbf{x}) = \mathbf{x}^\top \Sigma^{-1} \boldsymbol{\mu}_k - \frac{1}{2} \boldsymbol{\mu}_k^\top \Sigma^{-1} \boldsymbol{\mu}_k + \log \pi_k
$$

This is **linear in $\mathbf{x}$** — hence the name "Linear" Discriminant Analysis.

### Parameters to Estimate

From training data, we estimate:

1. **Class priors:** $\hat{\pi}_k = n_k / n$ (proportion of samples in class $k$)
2. **Class means:** $\hat{\boldsymbol{\mu}}_k = \frac{1}{n_k} \sum_{i: y_i = k} \mathbf{x}_i$
3. **Shared covariance:** $\hat{\Sigma} = \frac{1}{n - K} \sum_{k=1}^{K} \sum_{i: y_i = k} (\mathbf{x}_i - \hat{\boldsymbol{\mu}}_k)(\mathbf{x}_i - \hat{\boldsymbol{\mu}}_k)^\top$


In [ ]:
# Step 0: Generate sample data
import numpy as np

np.random.seed(42)

# Create two classes with different means but we'll assume same covariance
n_samples_per_class = 100
d = 2  # 2 features for easy visualisation

# Class 0: centred at (0, 0)
mean_0 = np.array([0, 0])
# Class 1: centred at (2, 2)
mean_1 = np.array([2, 2])

# Shared covariance matrix
cov = np.array([[1.0, 0.5],
                [0.5, 1.0]])

X_0 = np.random.multivariate_normal(mean_0, cov, n_samples_per_class)
X_1 = np.random.multivariate_normal(mean_1, cov, n_samples_per_class)

X = np.vstack([X_0, X_1])
y = np.array([0] * n_samples_per_class + [1] * n_samples_per_class)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Classes: {np.unique(y)}")

In [ ]:
# Step 1: Estimate class priors (π_k = n_k / n)
classes = np.unique(y)
n = len(y)

priors = {}
for k in classes:
    n_k = np.sum(y == k)
    priors[k] = n_k / n

print("Class priors:")
for k, pi_k in priors.items():
    print(f"  π_{k} = {pi_k:.3f}")

In [ ]:
# Step 2: Estimate class means (μ_k = mean of samples in class k)
means = {}
for k in classes:
    X_k = X[y == k]  # Get all samples belonging to class k
    means[k] = np.mean(X_k, axis=0)

print("Class means:")
for k, mu_k in means.items():
    print(f"  μ_{k} = {mu_k}")

In [ ]:
# Step 3: Estimate shared covariance matrix (pooled within-class covariance)
# Σ = (1 / (n - K)) * Σ_k Σ_i (x_i - μ_k)(x_i - μ_k)^T

K = len(classes)
d = X.shape[1]

# Accumulate the sum of outer products
cov_sum = np.zeros((d, d))
for k in classes:
    X_k = X[y == k]
    mu_k = means[k]
    # Centre the data for this class
    X_centred = X_k - mu_k
    # Add contribution to covariance sum
    cov_sum += X_centred.T @ X_centred

# Divide by (n - K) for unbiased estimate
shared_cov = cov_sum / (n - K)

print("Shared covariance matrix:")
print(shared_cov)

In [ ]:
# Step 4: Compute discriminant function and predict
# δ_k(x) = x^T Σ^{-1} μ_k - (1/2) μ_k^T Σ^{-1} μ_k + log(π_k)

# Compute inverse of shared covariance
cov_inv = np.linalg.inv(shared_cov)

def lda_discriminant(x, mu_k, cov_inv, pi_k):
    """Compute LDA discriminant function for a single class."""
    term1 = x @ cov_inv @ mu_k
    term2 = 0.5 * mu_k @ cov_inv @ mu_k
    term3 = np.log(pi_k)
    return term1 - term2 + term3

def lda_predict(X, means, cov_inv, priors, classes):
    """Predict class labels for all samples."""
    n_samples = X.shape[0]
    predictions = np.zeros(n_samples, dtype=int)
    
    for i, x in enumerate(X):
        # Compute discriminant for each class
        scores = {}
        for k in classes:
            scores[k] = lda_discriminant(x, means[k], cov_inv, priors[k])
        # Predict class with highest score
        predictions[i] = max(scores, key=scores.get)
    
    return predictions

# Make predictions
y_pred = lda_predict(X, means, cov_inv, priors, classes)

# Calculate accuracy
accuracy = np.mean(y_pred == y)
print(f"LDA Accuracy: {accuracy:.2%}")

In [ ]:
# Step 5: Verify against scikit-learn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sklearn_lda = LinearDiscriminantAnalysis()
sklearn_lda.fit(X, y)
y_pred_sklearn = sklearn_lda.predict(X)

sklearn_accuracy = np.mean(y_pred_sklearn == y)
print(f"scikit-learn LDA Accuracy: {sklearn_accuracy:.2%}")
print(f"Predictions match: {np.all(y_pred == y_pred_sklearn)}")

## Quadratic Discriminant Analysis (QDA)

QDA relaxes the LDA assumption: each class $k$ can have its **own covariance matrix** $\Sigma_k$. This allows for more flexible, curved decision boundaries.

### The Model

For each class $k$, the class-conditional density is:

$$
p(\mathbf{x} \mid y = k) = \frac{1}{(2\pi)^{d/2} |\Sigma_k|^{1/2}} \exp\left( -\frac{1}{2} (\mathbf{x} - \boldsymbol{\mu}_k)^\top \Sigma_k^{-1} (\mathbf{x} - \boldsymbol{\mu}_k) \right)
$$

Note: now we have $\Sigma_k$ instead of a shared $\Sigma$.

### The Discriminant Function

Taking the log, we get the **quadratic discriminant function**:

$$
\delta_k(\mathbf{x}) = -\frac{1}{2} \log |\Sigma_k| - \frac{1}{2} (\mathbf{x} - \boldsymbol{\mu}_k)^\top \Sigma_k^{-1} (\mathbf{x} - \boldsymbol{\mu}_k) + \log \pi_k
$$

This is **quadratic in $\mathbf{x}$** because of the $\mathbf{x}^\top \Sigma_k^{-1} \mathbf{x}$ term (which doesn't cancel out since each class has a different $\Sigma_k$).

### Parameters to Estimate

From training data, we estimate:

1. **Class priors:** $\hat{\pi}_k = n_k / n$
2. **Class means:** $\hat{\boldsymbol{\mu}}_k = \frac{1}{n_k} \sum_{i: y_i = k} \mathbf{x}_i$
3. **Class-specific covariances:** $\hat{\Sigma}_k = \frac{1}{n_k - 1} \sum_{i: y_i = k} (\mathbf{x}_i - \hat{\boldsymbol{\mu}}_k)(\mathbf{x}_i - \hat{\boldsymbol{\mu}}_k)^\top$


In [ ]:
# Step 0: Generate sample data with DIFFERENT covariances per class
np.random.seed(42)

n_samples_per_class = 100

# Class 0: centred at (0, 0) with one covariance
mean_0 = np.array([0, 0])
cov_0 = np.array([[1.0, 0.2],
                  [0.2, 1.0]])

# Class 1: centred at (2, 2) with a DIFFERENT covariance
mean_1 = np.array([2, 2])
cov_1 = np.array([[1.5, -0.8],
                  [-0.8, 1.5]])

X_0 = np.random.multivariate_normal(mean_0, cov_0, n_samples_per_class)
X_1 = np.random.multivariate_normal(mean_1, cov_1, n_samples_per_class)

X_qda = np.vstack([X_0, X_1])
y_qda = np.array([0] * n_samples_per_class + [1] * n_samples_per_class)

print(f"X shape: {X_qda.shape}")
print(f"Classes have DIFFERENT covariances - QDA should work better here")

In [ ]:
# Steps 1-2: Estimate class priors and means (same as LDA)
classes_qda = np.unique(y_qda)
n_qda = len(y_qda)

priors_qda = {}
means_qda = {}

for k in classes_qda:
    n_k = np.sum(y_qda == k)
    priors_qda[k] = n_k / n_qda
    means_qda[k] = np.mean(X_qda[y_qda == k], axis=0)

print("Class priors:")
for k, pi_k in priors_qda.items():
    print(f"  π_{k} = {pi_k:.3f}")

print("\nClass means:")
for k, mu_k in means_qda.items():
    print(f"  μ_{k} = {mu_k}")

In [ ]:
# Step 3: Estimate class-specific covariance matrices (KEY DIFFERENCE from LDA)
# Σ_k = (1 / (n_k - 1)) * Σ_i (x_i - μ_k)(x_i - μ_k)^T  for samples in class k

covariances_qda = {}

for k in classes_qda:
    X_k = X_qda[y_qda == k]
    mu_k = means_qda[k]
    n_k = len(X_k)
    
    # Centre the data
    X_centred = X_k - mu_k
    
    # Compute covariance for this class (unbiased estimate)
    covariances_qda[k] = (X_centred.T @ X_centred) / (n_k - 1)

print("Class-specific covariance matrices:")
for k, cov_k in covariances_qda.items():
    print(f"\nΣ_{k} =")
    print(cov_k)

In [ ]:
# Step 4: Compute quadratic discriminant function and predict
# δ_k(x) = -0.5 * log|Σ_k| - 0.5 * (x - μ_k)^T Σ_k^{-1} (x - μ_k) + log(π_k)

def qda_discriminant(x, mu_k, cov_k, cov_k_inv, pi_k):
    """Compute QDA discriminant function for a single class."""
    diff = x - mu_k
    
    term1 = -0.5 * np.log(np.linalg.det(cov_k))  # log determinant penalty
    term2 = -0.5 * (diff @ cov_k_inv @ diff)      # Mahalanobis distance
    term3 = np.log(pi_k)                          # prior bonus
    
    return term1 + term2 + term3

def qda_predict(X, means, covariances, priors, classes):
    """Predict class labels for all samples."""
    n_samples = X.shape[0]
    predictions = np.zeros(n_samples, dtype=int)
    
    # Precompute inverses
    cov_invs = {k: np.linalg.inv(covariances[k]) for k in classes}
    
    for i, x in enumerate(X):
        scores = {}
        for k in classes:
            scores[k] = qda_discriminant(
                x, means[k], covariances[k], cov_invs[k], priors[k]
            )
        predictions[i] = max(scores, key=scores.get)
    
    return predictions

# Make predictions
y_pred_qda = qda_predict(X_qda, means_qda, covariances_qda, priors_qda, classes_qda)

# Calculate accuracy
accuracy_qda = np.mean(y_pred_qda == y_qda)
print(f"QDA Accuracy: {accuracy_qda:.2%}")

In [ ]:
# Step 5: Verify against scikit-learn
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

sklearn_qda = QuadraticDiscriminantAnalysis()
sklearn_qda.fit(X_qda, y_qda)
y_pred_sklearn_qda = sklearn_qda.predict(X_qda)

sklearn_accuracy_qda = np.mean(y_pred_sklearn_qda == y_qda)
print(f"scikit-learn QDA Accuracy: {sklearn_accuracy_qda:.2%}")
print(f"Predictions match: {np.all(y_pred_qda == y_pred_sklearn_qda)}")

In [ ]:
# Step 6: Compare LDA vs QDA on this dataset
# Since classes have different covariances, QDA should have an advantage

sklearn_lda_compare = LinearDiscriminantAnalysis()
sklearn_lda_compare.fit(X_qda, y_qda)
lda_accuracy_compare = sklearn_lda_compare.score(X_qda, y_qda)

print("Comparison on data with DIFFERENT class covariances:")
print(f"  LDA Accuracy: {lda_accuracy_compare:.2%}")
print(f"  QDA Accuracy: {sklearn_accuracy_qda:.2%}")
print(f"\nQDA performs better because it can model the different covariance structures.")